In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

C:\Users\karin\AppData\Local\Temp\ipykernel_25196\4219097770.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def cargar_dataframes(ruta_csv1, ruta_csv2, ruta_csv3):
    """Carga los dataframes desde archivos CSV.

    Args:
    - ruta_flight (str): La ruta al archivo CSV de Customer Flight Activity.
    - ruta_loyalty (str): La ruta al archivo CSV de Customer Loyalty History.

    Returns:
    - df_flight (DataFrame): El dataframe cargado desde el archivo CSV de Customer Flight Activity.
    - df_loyalty (DataFrame): El dataframe cargado desde el archivo CSV de Customer Loyalty History.
    """
    df_clientes = pd.read_csv(ruta_csv1, index_col=0).reset_index()
    df_productos = pd.read_csv(ruta_csv2, index_col=0, on_bad_lines="warn").reset_index()
    df_ventas = pd.read_csv(ruta_csv3, index_col=0).reset_index()
    
    return df_clientes, df_productos, df_ventas


cargar_dataframes("files/clientes.csv","files/productos.csv", "files/ventas.csv")

def union_dataframes(df1, df2, df3):
    """
    Unimos los DataFrames utilizando la función merge de pandas.
    Args:
    - df1 (DataFrame): El primer DataFrame a unir.
    - df2 (DataFrame): El segundo DataFrame a unir.
    - df3 (DataFrame): El tercer DataFrame a unir.

    Returns:
    - df (DataFrame): El DataFrame resultante de la unión de los tres DataFrames.
    """
    # Unir los DataFrames df1 y df2 utilizando merge
    df_temp = pd.merge(df1, df2, left_index=True, right_index=True)

    # Unir el DataFrame temporal con df3
    df = pd.merge(df_temp, df3, left_index=True, right_index=True)

    return df

df_clientes, df_productos, df_ventas = cargar_dataframes("files/clientes.csv","files/productos.csv", "files/ventas.csv")
df = union_dataframes(df_clientes, df_productos, df_ventas)
df.head()

,id,first_name,last_name,email,gender,City,Country,Address,index,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing,A1,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate,mozzarella fresca y albahaca.,723,A1,2023-11-22,2,17.98
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,Risotto cremoso con champiñones frescos,una delicia italiana para disfrutar en casa.,498,C3,2023-11-21,1,5.49
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...,mascarpone y cacao en polvo.,121,D4,2023-11-20,3,32.97
3,4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road,D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...,un regalo tradicional en Italia.,885,L12,2023-11-19,1,6.49
4,5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle,E5,Orecchiette,Productos Secos,4.29,Italia,Pequeñas pastas con forma de orecchiette ideal...,NaN,347,Q17,2023-11-18,2,7.98


In [3]:
def exploracion_dataframe(dataframe, columna_control):
    """
    Realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre duplicados,
    valores nulos, tipos de datos, valores únicos para columnas categóricas y estadísticas descriptivas
    para columnas categóricas y numéricas, agrupadas por la columna de control.

    Parámetros:
    - dataframe (DataFrame): El DataFrame que se va a explorar.
    - columna_control (str): El nombre de la columna que se utilizará como control para dividir el DataFrame.

    Returns: 
    No devuelve nada directamente, pero imprime en la consola la información exploratoria.
    """
    
    print(f"Los duplicados que tenemos en el conjunto de datos son: {dataframe.duplicated().sum()}")
    print("\n ..................... \n")
    
    
    # generamos un DataFrame para los valores nulos
    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Los tipos de las columnas son:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    
    
    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()).head())    
    
    # como estamos en un problema de A/B testing y lo que realmente nos importa es comparar entre el grupo de control y el de test, los principales estadísticos los vamos a sacar de cada una de las categorías
    
    for categoria in dataframe[columna_control].unique():
        
        dataframe_filtrado = dataframe[dataframe[columna_control] == categoria]
    
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas categóricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe(include = "O").T)
        
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas numéricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe().T)
        
exploracion_dataframe(df, "ID")

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
email,9.523810
gender,9.523810
City,19.047619
Country,19.047619
Address,4.761905
Descripción,4.761905



 ..................... 

Los tipos de las columnas son:


,tipo_dato
id,int64
first_name,object
last_name,object
email,object
gender,object
City,object
Country,object
Address,object
index,object
ID,object



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna FIRST_NAME tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
id,1.0,20.00,NaN,20.00,20.00,20.00,20.00,20.00
Categoría,1.0,6.75,NaN,6.75,6.75,6.75,6.75,6.75
ID_Cliente,1.0,987.00,NaN,987.00,987.00,987.00,987.00,987.00
Cantidad,1.0,1.00,NaN,1.00,1.00,1.00,1.00,1.00
Total,1.0,6.99,NaN,6.99,6.99,6.99,6.99,6.99



 ..................... 

Los principales estadísticos de las columnas categóricas para el CICERI E TRIA son: 


,count,unique,top,freq
first_name,1,1,Frasquito,1
last_name,1,1,Skeemor,1
email,1,1,fskeemork@t-online.de,1
gender,1,1,Male,1
City,1,1,Murcia,1
Country,1,1,Spain,1
Address,1,1,46 Golf View Alley,1
index,1,1,GG33,1
ID,1,1,Ciceri e Tria,1
Nombre_Producto,1,1,Platos Preparados,1



 ..................... 

Los principales estadísticos de las columnas numéricas para el CICERI E TRIA son: 


,count,mean,std,min,25%,50%,75%,max
id,1.0,21.00,NaN,21.00,21.00,21.00,21.00,21.00
Categoría,1.0,5.89,NaN,5.89,5.89,5.89,5.89,5.89
ID_Cliente,1.0,234.00,NaN,234.00,234.00,234.00,234.00,234.00
Cantidad,1.0,3.00,NaN,3.00,3.00,3.00,3.00,3.00
Total,1.0,26.97,NaN,26.97,26.97,26.97,26.97,26.97


In [4]:
def configurar_visualizacion():
    """Configura la visualización de pandas para mostrar todas las columnas y formato de los números."""
    pd.set_option('display.max_columns', None)
    pd.set_option('display.float_format', '{:.2f}'.format)
    

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 0 to 20
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               21 non-null     int64  
 1   first_name       21 non-null     object 
 2   last_name        21 non-null     object 
 3   email            19 non-null     object 
 4   gender           19 non-null     object 
 5   City             17 non-null     object 
 6   Country          17 non-null     object 
 7   Address          20 non-null     object 
 8   index            21 non-null     object 
 9   ID               21 non-null     object 
 10  Nombre_Producto  21 non-null     object 
 11  Categoría        21 non-null     float64
 12  Precio           21 non-null     object 
 13  Origen           21 non-null     object 
 14  Descripción      20 non-null     object 
 15  ID_Cliente       21 non-null     int64  
 16  ID_Producto      21 non-null     object 
 17  Fecha_Venta      21 non

In [6]:
def imputar_valores_nulos(df, columnas_desconocido):
    """Imputa valores nulos en una lista de columnas del DataFrame utilizando 'fillna' con 'Unknown'.

    Args:
    - df (DataFrame): El DataFrame en el que se imputarán los valores nulos.
    - columnas_desconocido (list): Lista de nombres de las columnas en las que se imputarán los valores nulos.
    """
    # Iterar sobre la lista de columnas a las que se les cambiarán los nulos por "Unknown"
    
    for columna in columnas_desconocido:
        # Reemplazar los nulos por el valor "Unknown" para cada una de las columnas de la lista
        df[columna] = df[columna].fillna("Unknown")
        
    # Comprobar si quedan nulos en las columnas categóricas
    print("Después del reemplazo usando 'fillna', quedan los siguientes nulos:")
    print(df[columnas_desconocido].isnull().sum())
    
    return df

columnas_desconocido = ["email","gender","City","Country","Address", "Descripción"]
imputar_valores_nulos(df,columnas_desconocido)

Después del reemplazo usando 'fillna', quedan los siguientes nulos:
email          0
gender         0
City           0
Country        0
Address        0
Descripción    0
dtype: int64


,id,first_name,last_name,email,gender,City,Country,Address,index,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing,A1,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate,mozzarella fresca y albahaca.,723,A1,2023-11-22,2,17.98
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,Risotto cremoso con champiñones frescos,una delicia italiana para disfrutar en casa.,498,C3,2023-11-21,1,5.49
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,Unknown,Spain,0 Corry Crossing,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...,mascarpone y cacao en polvo.,121,D4,2023-11-20,3,32.97
3,4,Filmer,Eirwin,feirwin3@intel.com,Unknown,Leon,Spain,5 American Ash Road,D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...,un regalo tradicional en Italia.,885,L12,2023-11-19,1,6.49
4,5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle,E5,Orecchiette,Productos Secos,4.29,Italia,Pequeñas pastas con forma de orecchiette ideal...,Unknown,347,Q17,2023-11-18,2,7.98
5,6,Stacy,Minchinton,sminchinton5@ameblo.jp,Male,Palma De Mallorca,Spain,4 Meadow Vale Way,F6,Polenta Tradicional,Harinas,3.75,Italia,Harina de maíz italiana para preparar la tradi...,perfecta como acompañamiento.,619,T20,2023-11-17,1,7.89
6,7,Harv,Sandes,hsandes6@who.int,Male,Lleida,Spain,20 Menomonie Circle,G7,Mozzarella di Bufala,Productos Lácteos,9.25,Italia,Mozzarella fresca elaborada con leche de búfala,suave y deliciosamente cremosa.,812,Y25,2023-11-16,2,13.50
7,8,Aubree,Jirsa,Unknown,Female,Santander,Unknown,8294 Schurz Court,J10,Arroz Arborio,Productos Secos,4.89,Italia,Arroz de grano corto perfecto para risottos it...,absorbe los sabores de forma exquisita.,170,U21,2023-11-15,1,5.75
8,9,Tymon,Darragh,tdarragh8@free.fr,Male,Unknown,Spain,41 Trailsway Way,K11,Café Espresso,Café,6.49,Italia,Café espresso italiano con un aroma y sabor in...,la esencia de la cultura del café en Italia.,29,Z26,2023-11-14,2,14.98
9,10,Igor,Dwelley,idwelley9@flickr.com,Male,Lugo,Spain,Unknown,L12,Sardinas en Aceite de Oliva,Pescados en Conserva,3.99,Italia,Sardinas frescas en aceite de oliva,un bocado delicioso y saludable.,567,AA27,2023-11-13,1,7.49


In [7]:
def eliminar_columnas(dataframe, columnas):
    """Elimina columnas del DataFrame.

    Args:
    - dataframe (DataFrame): El DataFrame del que se eliminarán las columnas.
    - columnas (list): La lista de nombres de las columnas que se eliminarán.
    """
    dataframe.drop(columns=columnas, inplace=True)

columnas_eliminar = ["id", "index", "Precio", "Country"]
eliminar_columnas(df, columnas_eliminar)


In [8]:
def transformar_nombres_columnas(dataframe):
    """Transforma los nombres de las columnas del DataFrame.

    Args:
    - dataframe (DataFrame): El DataFrame cuyos nombres de columnas se transformarán.
    """
    nuevas_columnas = [col.replace(" ", '_').lower() for col in dataframe.columns]
    dataframe.columns = nuevas_columnas
    dataframe.rename(columns={ 'nombre_producto': 'clasificacion_plato', 'id': 'nombre_producto',}, inplace=True)
    
    return dataframe

transformar_nombres_columnas(df)


<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 0 to 20
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   first_name           21 non-null     object 
 1   last_name            21 non-null     object 
 2   email                21 non-null     object 
 3   gender               21 non-null     object 
 4   city                 21 non-null     object 
 5   address              21 non-null     object 
 6   nombre_producto      21 non-null     object 
 7   clasificacion_plato  21 non-null     object 
 8   categoría            21 non-null     float64
 9   origen               21 non-null     object 
 10  descripción          21 non-null     object 
 11  id_cliente           21 non-null     int64  
 12  id_producto          21 non-null     object 
 13  fecha_venta          21 non-null     object 
 14  cantidad             21 non-null     int64  
 15  total                21 non-null     float64
dt